In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import argparse
from trainer.RetrievalTrainer import trainer
from datasets.ScannetDataset import ScannetDataset
from datasets.CategoryTestTimeDataset import *
from datasets.ScannetTestTimeDataset import *
from datasets.Reader import *

from test_time.FeatureExtractor import FeatureExtractor
from test_time.RetrievalModule import RetrievalModule
from test_time.PoseEstimator import PoseEstimator

from utils.logger import logger
from utils.ckpts import load_checkpoint, save_checkpoint
import utils.transform_estimation as te
from utils.eval import find_nn_gpu
from utils.retrieval import *
from utils.visualize import *
from utils.preprocess import random_rotation, load_norm_pc, apply_transform
from utils.symmetry import *
from utils.eval_pose import *
from utils.Info.Scan2cadInfo import Scan2cadInfo
from utils.Info.CADLib import *

from model import load_model, fc

from scipy.spatial.distance import pdist, squareform
from sklearn.cluster import KMeans
import matplotlib
import time

import MinkowskiEngine as ME

torch.manual_seed(0)
torch.cuda.manual_seed(0)

class Config():
    def __init__(self):

        self.root = "/scannet/ShapeNetCore.v2.PC15k"
        self.scan2cad_root = "/scannet/crop_scan2cad_filter/data"
        self.cad_root = "/scannet/ShapeNetCore.v2.PC15k"
        self.catid = "03001627"#"04379243"
        self.voxel_size = 0.03
        self.dim = [1024, 512,  256]
        self.embedding = "identity"
        self.resume = "./ckpts/scannet_table_pose_001_usedLib_bestMatch_Best"
        self.model = "ResUNetBN2C"
        self.model_n_out = 16
        self.normalize_feature = True
        self.conv1_kernel_size = 3
        self.bn_momentum = 0.05
        self.nn_max_n = 500
        self.lib_path = "./CadLib"
        self.scan2cad_dict = "/scannet/scan2cad_download_link/unique_cads.csv"
        self.annotation_dir = "/scannet/scan2cad_download_link"
        self.device = torch.device("cuda")
        #self.table_path = "/scannet/tables/04379243_full.npy"

config = Config()


def chamfer_max(pc0, pc1):
    
    pc0 = torch.Tensor(pc0).cuda()
    pc1 = torch.Tensor(pc1).cuda()
    
    pc0 = pc0[None,:,:]
    pc1 = pc1[:,None,:]
    delta = pc0 - pc1
    dist = delta.norm(dim=2)
    return max(dist.min(0)[0].max(), dist.min(1)[0].max())

def N_rot(num, pc, thres):

    errors = []
    for i in range(1, int(num/2)+1):
        T = np.eye(4)
        T[:3,:3] = transforms3d.euler.euler2mat(0, i*(2*np.pi)/num, 0)

        pc_rot = apply_transform(pc, T)

        error = chamfer_max(pc, pc_rot)
        
        if error > thres:
            return False
        
        errors.append(error)
        #Jvisualize([pc, pc_rot], ["RED", "GREEN"])
    
    return True



def labeling(root, catid, split):

    lib = os.path.join(root, catid, split)
    names = os.listdir(lib)
    names.sort()
    f = open("./config/{}_{}_rot_sym_label.txt".format(catid, split), "w").close()

    thres = 0.1
    for idx, name in enumerate(names):
        pc = load_norm_pc(os.path.join(lib, name), 4000)

        for num in [12, 8, 6, 4, 3, 2, 1]:
            if N_rot(num, pc, thres):
                f = open("./config/{}_{}_rot_sym_label.txt".format(catid, split), "a")
                f.writelines("{} {}\n".format(name, num))
                f.close()

                print("{} {}".format(name, num))
                break

def labeling_with_pathes(pathes, catid, save_name):


    f = open("./config/{}_{}_rot_sym_label.txt".format(catid, save_name), "w").close()

    thres = 0.1
    for idx, name in enumerate(pathes):
        pc = load_norm_pc(name, 4000)

        for num in [12, 8, 6, 4, 3, 2, 1]:
            if N_rot(num, pc, thres):
                f = open("./config/{}_{}_rot_sym_label.txt".format(catid, save_name), "a")
                f.writelines("{} {}\n".format(name, num))
                f.close()

                print("{} {}".format(name, num))
                break

In [3]:
root = "/zty-vol/data/ShapeNetCore.v2.PC15k"
catid = "03001627"

labeling(root, catid, "val")
#compile_label()

1006be65e7bc937e9141f9b58470d646.npy 1
103a60f3b09107df2da1314e036b435e.npy 1
106e9487a1d47ff1a09cb462b3e723e2.npy 1
107caefdad02cf1c8ab8e68cb52baa6a.npy 1
10a1783f635b3fc181dff5c2e57ad46e.npy 1
11347c7e8bc5881775907ca70d2973a4.npy 1
114f72b38dcabdf0823f29d871e57676.npy 1
1157d8d6995da5c0290d57214c8512a4.npy 1
1190af00b6c86c99c3bd24f986301745.npy 1
11b7c86fc42306ec7e7e25239e7b8f85.npy 1
11d3fc4092e616a7a6fee8e2140acec9.npy 1
11e28120789c20abc8687ff9b0b4e4ac.npy 1
177849848dc83fb9cb85ba5866080618.npy 1
182c47283b90d8b2d85934de5898f0b.npy 1
195c379defdff8b81dff5c2e57ad46e.npy 1
19a3ac2ec0312e13c8687ff9b0b4e4ac.npy 1
1ace72a88565df8e56bd8571ad86331a.npy 1
1be38f2624022098f71e06115e9c3b3e.npy 1
1bf710535121b17cf453cc5da9731a22.npy 1
1c2cad4df1697103a5139737ddc33fc8.npy 1
1c3f1a9cea91359c4c3e19c2c67c262f.npy 1
1cf77ee00faa6de7fa6450cce25dc4cb.npy 1
1e3fba4500d20bb49b9f2eb77f5e247e.npy 1
1eb2e372a204a61153baab6c8235f5db.npy 1
1eb5613aa22df87b8ef9327f5d5c024d.npy 1
1ee30d218c8c730ecb01bc908e8

5390dede41d523f71a782a4379556c7.npy 1
539dd50ce4d5fd97febad4f49b26ec52.npy 1
53eaa7cca72c84f6cacd67ce2c72c02e.npy 1
541746ddb47aa2af4e186c8346f12e6.npy 1
5457691268846b073ca87feffb4fd9a0.npy 1
5482bda23163df4f30e916fac260c672.npy 1
55dadd6acaa2a0c0dab72035db8eafd.npy 1
568050037ad958625ea33df4b8846880.npy 1
56bd6bcf5034c7e2f9b9977a2406713a.npy 1
57285ae595c5cc67eea9006d9d8918a7.npy 1
57b4898f5b94a5a89d297e74104d3ac3.npy 1
584ab19b5df1fe697daabf84de73fb1d.npy 1
58616932b0e7ab6d78b03575bb54dfd4.npy 1
587d174c5f2b19c1d7e48afa5cc5db41.npy 1
588ca1d34f93eddd8ee30571b0da74e4.npy 1
58a7b826ed562b7bb0957d845ac33749.npy 1
58e6fa617a1bed1a20b64028d2efc877.npy 1
59ca6d57ddd091c26eda1c7568601317.npy 1
5a871d6613216ecebda72093f9b5aa73.npy 1
5ab321d70991c94d780bc2708a85ba9a.npy 1
5aea2a227125a81bccda8d28b44378b7.npy 1
5b0e833cf2ea465e42bd82e95587d8af.npy 1
5b1d0dad82acd6e5893104fdda835c64.npy 1
5c1c5dd72247cb6526428c7519676cd8.npy 1
5c660775f1c7b202855c835f93769fb8.npy 1
5cbd738436d0523d6996454765a5

a487bfde2aa50faa6d7bcadc05e0f4c2.npy 1
a4b14d0f3cf8a8d7b37a59fbad57ca4.npy 1
a4bf3b34bf49c5aa799b0b9a0d4a12cf.npy 1
a544d05ee3bf362d27e1ddec710c515b.npy 1
a592c3bf73afeb9294741031f62be2f6.npy 1
a5eea0cf3865da9749fb53621b8edde7.npy 1
a631fb1b054697dcfd8aaf214a1df4be.npy 1
a63d7b6e62333b23ddc809959e304d48.npy 1
a64bc6079afa241f762ec387b772e9e1.npy 1
a65cd22689f14147f155d75bbf62b80.npy 1
a66d78cbf3603e8bba9df3ea0397b1a6.npy 1
a69c999cd76e29862f8c68dc62edc38.npy 1
a749c31cee6965f33dc9e0e58bb697d3.npy 1
a7e33746f41cd181e258d1c3e7302e7b.npy 1
a801ccbf4985a57f17adebabcec6b7e.npy 1
a85e81c62bed11ea7f4e21263a28c500.npy 1
a873a489386dc825ed843e67f2e2e9d.npy 1
a8da22b87a249bc9c9bfaa062f2e9d4c.npy 1
a9053230c8e02442c8687ff9b0b4e4ac.npy 1
a919152f07e9e0d0fdef1c01cbd4ae0c.npy 1
a95b5ea73cf0c9bc8e72c17826417b99.npy 1
a98b128938b56846ee316b26c8d85c48.npy 1
a9eae610d9ab7c40a7d147d66b8f507d.npy 1
aa154024474ad7d138a720835dbeef20.npy 1
aa412e95e1f9c047d9b53420a5458c53.npy 1
ad7a6efb5aa6c289810da4adc58441

f2c1f929f451753de4ee93147efa8cb1.npy 1
f3499b524c5dbbefa23949c21eddef76.npy 1
f35645e0b1141804a57b3bec78e5d1b3.npy 1
f4268a28d2a837a1167c009da6daa010.npy 1
f4c6c927df8edb62c7623e33dc7256f2.npy 1
f4db2e2e90e83c759b9f2eb77f5e247e.npy 1
f4e355f3c1acc9f01ec783b810d76c4a.npy 1
f64aa038d1e21e0ad2ebe04c523a4739.npy 1
f68c32ea1e6cecd3c48d90eef8384210.npy 1
f9d050a51da1f3ed30bbd4cddd04c77b.npy 1
fa7f42b395c3cfce520ab6214a789faf.npy 1
fac321238dc8637eaa7cb30470f3273c.npy 1
fb42c4a8cf7b6d36ec89626a4213fd07.npy 1
fbfdc8eed6598437b2bb75885cfc44.npy 1
fc0e86a20a9606493cd6abbbeca4e2ad.npy 1
fc369b19cfb50322492d9da2668ec34c.npy 1
fc557cf617e03564da733bb42d274ff9.npy 1
fc7b535acdde11f4aa3e869226c9586e.npy 1
fdc7f288b8c65494a7101992f797b6a.npy 1
fde328d91eee1480fc0035da39bd5e1.npy 1
fe9467d48dc6b3491dfc750afa3f442.npy 1
ff3581996365bdddc3bd24f986301745.npy 1
ff793ca32407b2022a54a8462d1ffe9d.npy 1
ffd258571807e6425b1205fcf56bb774.npy 1
u1e22cc04-7c4d-4ed5-bda3-8ff8067f22ee.npy 1
ue639c33f-d415-458c-8ff8-

In [12]:
scan2cad_info.UsedObjPath

['/scannet/ShapeNetCore.v2.PC15k/03001627/val/7729a6ad96985f4ed1ccbd5d84e5bc86.npy',
 '/scannet/ShapeNetCore.v2.PC15k/03001627/train/561d50862a2cb4aabc19762eaa7ba40f.npy',
 '/scannet/ShapeNetCore.v2.PC15k/03001627/test/6e46e4cce527abc114d449899f34bd9d.npy',
 '/scannet/ShapeNetCore.v2.PC15k/03001627/train/45214e3010f8c3ddafa9c337f30be0ab.npy',
 '/scannet/ShapeNetCore.v2.PC15k/03001627/train/95d082c78ea0b1befe52ffd0e748a1ab.npy',
 '/scannet/ShapeNetCore.v2.PC15k/03001627/train/3211fc4f31b15964aee892adc836ce83.npy',
 '/scannet/ShapeNetCore.v2.PC15k/03001627/test/764866604b035caacd0362ae35d1beb4.npy',
 '/scannet/ShapeNetCore.v2.PC15k/03001627/train/8ab6783b1dfbf3a8a5d9ad16964840ab.npy',
 '/scannet/ShapeNetCore.v2.PC15k/03001627/train/6f36520144753550f91663a74ccd2338.npy',
 '/scannet/ShapeNetCore.v2.PC15k/03001627/train/49434684366778973002761e7a3ba3bd.npy',
 '/scannet/ShapeNetCore.v2.PC15k/03001627/train/b58b8a4646d14b7c202339ec5396045d.npy',
 '/scannet/ShapeNetCore.v2.PC15k/03001627/train

In [16]:
scan2cad_info = Scan2cadInfo(config.cad_root, config.scan2cad_root, config.catid, config.annotation_dir)


labeling_with_pathes(scan2cad_info.UsedObjPath, config.catid, "scan2cad")

/scannet/ShapeNetCore.v2.PC15k/03001627/val/7729a6ad96985f4ed1ccbd5d84e5bc86.npy 1
/scannet/ShapeNetCore.v2.PC15k/03001627/train/561d50862a2cb4aabc19762eaa7ba40f.npy 1
/scannet/ShapeNetCore.v2.PC15k/03001627/test/6e46e4cce527abc114d449899f34bd9d.npy 1
/scannet/ShapeNetCore.v2.PC15k/03001627/train/45214e3010f8c3ddafa9c337f30be0ab.npy 1
/scannet/ShapeNetCore.v2.PC15k/03001627/train/95d082c78ea0b1befe52ffd0e748a1ab.npy 1
/scannet/ShapeNetCore.v2.PC15k/03001627/train/3211fc4f31b15964aee892adc836ce83.npy 1
/scannet/ShapeNetCore.v2.PC15k/03001627/test/764866604b035caacd0362ae35d1beb4.npy 1
/scannet/ShapeNetCore.v2.PC15k/03001627/train/8ab6783b1dfbf3a8a5d9ad16964840ab.npy 1
/scannet/ShapeNetCore.v2.PC15k/03001627/train/6f36520144753550f91663a74ccd2338.npy 1
/scannet/ShapeNetCore.v2.PC15k/03001627/train/49434684366778973002761e7a3ba3bd.npy 1
/scannet/ShapeNetCore.v2.PC15k/03001627/train/b58b8a4646d14b7c202339ec5396045d.npy 1
/scannet/ShapeNetCore.v2.PC15k/03001627/train/9dfaa9b599fced446f1e07a

/scannet/ShapeNetCore.v2.PC15k/03001627/train/68151e7a9fcd8545851e82fe58f53ce1.npy 1
/scannet/ShapeNetCore.v2.PC15k/03001627/train/904ddb7762488b1b5dc3653f8341633a.npy 1
/scannet/ShapeNetCore.v2.PC15k/03001627/val/da7aeb7a73f298329940c34828c5731f.npy 1
/scannet/ShapeNetCore.v2.PC15k/03001627/train/b89952c6f2dbc1aea50b88953d263a42.npy 1
/scannet/ShapeNetCore.v2.PC15k/03001627/test/b5877df9f111bb91222f2fee97f8bdcd.npy 1
/scannet/ShapeNetCore.v2.PC15k/03001627/val/1eb2e372a204a61153baab6c8235f5db.npy 1
/scannet/ShapeNetCore.v2.PC15k/03001627/train/8c76b1db0a08653ffebad4f49b26ec52.npy 1
/scannet/ShapeNetCore.v2.PC15k/03001627/train/c9913221819588b519fb4103277a6b93.npy 1
/scannet/ShapeNetCore.v2.PC15k/03001627/val/a2bffa12e0737e66febad4f49b26ec52.npy 1
/scannet/ShapeNetCore.v2.PC15k/03001627/train/942a10a33418205819fb4103277a6b93.npy 1
/scannet/ShapeNetCore.v2.PC15k/03001627/train/357e2dd1512b96168e2b488ea5fa466a.npy 1
/scannet/ShapeNetCore.v2.PC15k/03001627/test/49a3b0242c13f92da6fee8e2140

/scannet/ShapeNetCore.v2.PC15k/03001627/train/ca764efc4e76a8b48d69616377752ecc.npy 1
/scannet/ShapeNetCore.v2.PC15k/03001627/train/71aa917674beaa2926d044baef0e5df2.npy 1
/scannet/ShapeNetCore.v2.PC15k/03001627/train/7220dcc0f3f95da3a84d309c7a35a478.npy 1
/scannet/ShapeNetCore.v2.PC15k/03001627/train/2a39dbeb52262b975babe43b460dfe6e.npy 1
/scannet/ShapeNetCore.v2.PC15k/03001627/train/c747e6ceb1a6faaa3074f48b99186254.npy 1
/scannet/ShapeNetCore.v2.PC15k/03001627/train/5008eca8deba6555ee229bc98ff4457e.npy 1
/scannet/ShapeNetCore.v2.PC15k/03001627/train/56e51afd9d5c5fa38b7a92edf72424a7.npy 1
/scannet/ShapeNetCore.v2.PC15k/03001627/test/b72342e210414024e3e472a115551ec9.npy 1
/scannet/ShapeNetCore.v2.PC15k/03001627/train/1f65075818c1d832c05575120a46cd3b.npy 1
/scannet/ShapeNetCore.v2.PC15k/03001627/train/dfa7aa13ee696833593ebeeedbff73b.npy 1
/scannet/ShapeNetCore.v2.PC15k/03001627/val/a592c3bf73afeb9294741031f62be2f6.npy 1
/scannet/ShapeNetCore.v2.PC15k/03001627/train/86a25532f255728fdaeb838

/scannet/ShapeNetCore.v2.PC15k/03001627/train/224775a0c7f8fec65780b8d333445e3e.npy 1
/scannet/ShapeNetCore.v2.PC15k/03001627/test/d6f2d44c693d2e857062f2d72cde5c95.npy 1
/scannet/ShapeNetCore.v2.PC15k/03001627/val/f64aa038d1e21e0ad2ebe04c523a4739.npy 1
/scannet/ShapeNetCore.v2.PC15k/03001627/train/d8774646afed0312732375ced502498.npy 1
/scannet/ShapeNetCore.v2.PC15k/03001627/train/5afc95ed2a26ce548644074928ab3f32.npy 1
/scannet/ShapeNetCore.v2.PC15k/03001627/train/bd5eb0bfad19c8d9cd431573238602d.npy 1
/scannet/ShapeNetCore.v2.PC15k/03001627/train/e955b63a4bd738857178717457aa5d20.npy 1
/scannet/ShapeNetCore.v2.PC15k/03001627/train/7824e41c86f9be776383f517a315e69e.npy 1
/scannet/ShapeNetCore.v2.PC15k/03001627/train/7a338e9b13ae5da3245e874042d8b8db.npy 1
/scannet/ShapeNetCore.v2.PC15k/03001627/test/68b88c0be088c21d5e0096fb2d3266a.npy 1
/scannet/ShapeNetCore.v2.PC15k/03001627/train/e39df7339552595febad4f49b26ec52.npy 1
/scannet/ShapeNetCore.v2.PC15k/03001627/test/cbc9014bb6ce3d902ff834514c92

/scannet/ShapeNetCore.v2.PC15k/03001627/test/e4b40369894a16ce6821a1e68ba5ebab.npy 1
/scannet/ShapeNetCore.v2.PC15k/03001627/train/79397634d362eaded96cac5d008c9fc3.npy 1
/scannet/ShapeNetCore.v2.PC15k/03001627/val/804c35699b163b7756bc5724b6282816.npy 1
/scannet/ShapeNetCore.v2.PC15k/03001627/train/7fa4db28bd803691c91adfee365fdcaf.npy 1
/scannet/ShapeNetCore.v2.PC15k/03001627/train/9b52e73f96d181969cd431573238602d.npy 1
/scannet/ShapeNetCore.v2.PC15k/03001627/val/511a03618c9bf2e7fa762d956b3074e4.npy 1
/scannet/ShapeNetCore.v2.PC15k/03001627/train/65c7a3d8fb95311c492d9da2668ec34c.npy 1
/scannet/ShapeNetCore.v2.PC15k/03001627/train/4bd7a49e63e0e97936e3b2fa8d1eb4eb.npy 1
/scannet/ShapeNetCore.v2.PC15k/03001627/test/317a2cc9f3b54a48283b00891f680579.npy 1
/scannet/ShapeNetCore.v2.PC15k/03001627/train/8b54af871a3ac77c492d9da2668ec34c.npy 1
/scannet/ShapeNetCore.v2.PC15k/03001627/train/6ade3daff4827379492d9da2668ec34c.npy 1
/scannet/ShapeNetCore.v2.PC15k/03001627/train/c5d880efc887f6f4f9111ef49

/scannet/ShapeNetCore.v2.PC15k/03001627/train/9de69e28a07c6e10592cc471d13e2037.npy 1
/scannet/ShapeNetCore.v2.PC15k/03001627/train/6f04104e178fb858f7628281ecb18112.npy 1
/scannet/ShapeNetCore.v2.PC15k/03001627/train/3515073c473f4ec94b3c42e318f3affc.npy 1
/scannet/ShapeNetCore.v2.PC15k/03001627/train/116a9cd5ac9a008ee8cb931b01d0a372.npy 1
/scannet/ShapeNetCore.v2.PC15k/03001627/train/c56bca4b7f353ad86debb0a33c851f8.npy 1
/scannet/ShapeNetCore.v2.PC15k/03001627/train/d323e6d19dc58526f2c0effc06a15c11.npy 1
/scannet/ShapeNetCore.v2.PC15k/03001627/train/d9bf326d8ae7a430a5a7dec3df261ad1.npy 1
/scannet/ShapeNetCore.v2.PC15k/03001627/train/2ca91e56bef8cb5034af953b663e921b.npy 1
/scannet/ShapeNetCore.v2.PC15k/03001627/train/edb86d102237488ce8f8c11a24c52ebb.npy 1
/scannet/ShapeNetCore.v2.PC15k/03001627/train/5ed72ca90351b0c4d5b24cafb84903c7.npy 1
/scannet/ShapeNetCore.v2.PC15k/03001627/train/4a0e7f1129a9172349bb1cbbf9a4206.npy 1
/scannet/ShapeNetCore.v2.PC15k/03001627/test/b3a9c49a1924f99815f855

In [14]:
len(scan2cad_info.UsedObjPath)

830